In [1]:
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import transformers
print(transformers.__version__)
from transformers.utils import send_example_telemetry
from transformers import TrainingArguments, Trainer, default_data_collator
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoConfig
from datasets import load_dataset, load_metric
from datasets import ClassLabel, Sequence
from sklearn.metrics import accuracy_score, f1_score

/home/ujjwal/miniconda3/envs/torch/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


4.36.1


/home/ujjwal/miniconda3/envs/torch/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ujjwal/miniconda3/envs/torch/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
data=pd.read_csv("NLPConversation.csv")
data=data.dropna()
n=len(data)
print(n)
N=list(range(n))
random.shuffle(N)
data=data.iloc[N][0:1000].reset_index(drop=True)
display(data)
print(data.columns.tolist())
print('context')
print(data.iloc[0,0])
print()
print('response')
print(data.iloc[0,1])

3508


,Context,Response
0,I have no self control over food. Most people ...,I would recommend removing yourself from the e...
1,I am in a high stress position for a tech comp...,Being in this position is tough. If seeking an...
2,I need help knowing how to deal with stress. W...,Our body reacts to stress typically by breathi...
3,My daughter didn't see her biological father f...,"Hi Dillon,I'm from Canada, so I don't know the..."
4,Every winter I find myself getting sad because...,"One theory is that instead of ""fighting"" your ..."
...,...,...
995,I find myself being very outgoing most of the ...,Would you feel more secure in conversations if...
996,"My ex-boyfriend, will not stop harassing and s...",The specific laws about this will vary from st...
997,I have been dating my ex-boyfriend’s cousin fo...,Does your boyfriend agree with the other peopl...
998,My boyfriend recently got a kitty. I hate cats...,Sorry for you and sorry for the cat because yo...


['Context', 'Response']
context
I have no self control over food. Most people stop when they've had enough, but I keep eating for the pleasure of it. Especially with sweets - I'm never done eating dessert.

response
I would recommend removing yourself from the environment you are in after you have finished eating. A simple walk around the block, calling up a friend and going to visit, or even going to another room in the house can help. If you find that you truly have no self control over food, I recommend attending Overeaters Anonymous; a 12 step group for those with compulsive issues related to food. I hope this helps.


In [3]:
data.Context[10]

"I'm  very depressed. How do I find someone to talk to?"

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

In [5]:
class CustomModel(AutoModelForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        self.config.max_position_embeddings = 512  # Set your desired size
        self.max_sequence_length = 136  # Set the same value for consistency
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                inputs_embeds=None, start_positions=None, end_positions=None, output_attentions=None,
                output_hidden_states=None, return_dict=None):

        # Check and truncate input_ids if necessary
        if input_ids is not None and input_ids.size(1) > self.max_sequence_length:
            input_ids = input_ids[:, :self.max_sequence_length]
            
        # Print the sizes of important tensors for debugging
        print("Input_ids size:", input_ids.size())
        print("Attention_mask size:", attention_mask.size() if attention_mask is not None else None)          

        # Continue with the rest of the forward method
        return super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            start_positions=start_positions,
            end_positions=end_positions,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
    
model_name = "roberta-base"
config = AutoConfig.from_pretrained(model_name)
model = CustomModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


In [6]:
config

RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.36.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [7]:
model

RobertaForCausalLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [8]:
def format_df(dataframe, tokenizer, max_sequence_length=512):
    formatted_data = []
    
    for _, row in dataframe.iterrows():
        input_text = f"{row['Context']} "
        
        if len(input_text) > max_sequence_length:
            input_text = input_text[:max_sequence_length]

        inputs = tokenizer(input_text, return_tensors="pt", max_length=max_sequence_length, truncation=True)    
 
        if 'Response' in row:
            answer_text = row['Response']
            answer_tokens = tokenizer(answer_text, return_tensors="pt")["input_ids"] 
        else:
            answer_text, answer_tokens = None, None

        formatted_data.append({
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            #'labels': answer_tokens,
        })

    return formatted_data

In [9]:
formatted_df = format_df(data, tokenizer)
print(formatted_df[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (599 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': tensor([[    0,   100,    33,   117,  1403,   797,    81,   689,     4,  1993,
            82,   912,    77,    51,   348,    56,   615,     6,    53,    38,
           489,  4441,    13,     5, 10483,     9,    24,     4, 17570,    19,
         29842,   111,    38,   437,   393,   626,  4441, 17927,     4,  1437,
             2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [10]:
class CustomDataset(Dataset):
    def __init__(self, formatted_data):
        self.formatted_data = formatted_data

    def __len__(self):
        return len(self.formatted_data)

    def __getitem__(self, idx):
        return {
            'input_ids': self.formatted_data[idx]['input_ids'].squeeze(),
            'attention_mask': self.formatted_data[idx]['attention_mask'].squeeze(), 
            #'labels': self.formatted_data[idx]['labels'].squeeze() if self.formatted_data[idx]['labels'] is not None else None,
            'start_positions': self.formatted_data[idx]['start_positions'] if 'start_positions' in self.formatted_data[idx] else None,
            'end_positions': self.formatted_data[idx]['end_positions'] if 'end_positions' in self.formatted_data[idx] else None
        }
    
n=len(formatted_df)
trainset = CustomDataset(formatted_df[0:n*4//5])
testset = CustomDataset(formatted_df[n*4//5:])

In [11]:
from torch.nn.utils.rnn import pad_sequence

def custom_collate_fn(batch):
    input_ids = pad_sequence([item['input_ids'] for item in batch], batch_first=True)
    attention_mask = pad_sequence([item['attention_mask'] for item in batch], batch_first=True)

    #labels = pad_sequence([item['labels'] for item in batch if item['labels'] is not None], batch_first=True) if any(item['labels'] is not None for item in batch) else None

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        #'labels': labels,
    }
        
batch_size = 16

train_dataloader = DataLoader(trainset, batch_size=batch_size, 
                              shuffle=True, collate_fn=custom_collate_fn)
test_dataloader = DataLoader(testset, batch_size=batch_size, 
                              shuffle=False, collate_fn=custom_collate_fn)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
model.to(device)
model.train()

RobertaForCausalLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [13]:
for batch in train_dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    #labels = batch['labels'].to(device) if batch['labels'] is not None else None

    outputs = model(input_ids=input_ids, attention_mask=attention_mask,)

In [14]:
model.eval()
all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # Get the predicted token IDs
        logits = outputs.logits
        predicted_ids = torch.argmax(logits, dim=-1)

        # Convert token IDs to tokens
        predicted_tokens = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

        # Extend the list of predictions
        all_predictions.extend(predicted_tokens)

In [15]:
from nltk.translate.bleu_score import corpus_bleu

test=data[n*4//5:]
test['predicted']=all_predictions
test=test[['Response','predicted']]
#test_df['answer_text']=test_df['answers'].apply(lambda x:x.get('text',[''])[0])
display(test)

references=test['Response'].tolist()
hypotheses =test['predicted'].tolist()

bleu_score = corpus_bleu(references, hypotheses)
#accuracy = accuracy_score(references, hypotheses)
#f1 = f1_score(references, hypotheses, average='micro')

print(f"BLEU Score: {bleu_score:.4f}")
#print(f"Accuracy: {accuracy:.4f}")
#print(f"F1 Score: {f1:.4f}")

/tmp/ipykernel_269686/1592975765.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted']=all_predictions


,Response,predicted
800,"Hi Attica,This is a question I think a lot of ...",My ex-boyfriend boyfriend and I lived together...
801,Do the two of you ever talk over why he does t...,My boyfriend is always saying he's done with m...
802,"Hi there, first off I have to commend you for ...",I'm a 40 year old male and having erection pro...
803,Awareness is the first step. Now that you kno...,I snap back and push people away. I need help ...
804,Thank you for reaching out! That is a great qu...,I feel like my time is going too fast
...,...,...
995,Would you feel more secure in conversations if...,I find myself being very outgoing most of the ...
996,The specific laws about this will vary from st...,"My ex-boyfriend, will not stop harassing and s..."
997,Does your boyfriend agree with the other peopl...,I have been dating my ex-boyfriend’s cousin fo...
998,Sorry for you and sorry for the cat because yo...,My boyfriend recently got a kitty. I hate cats...


BLEU Score: 0.0000


/home/ujjwal/miniconda3/envs/torch/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ujjwal/miniconda3/envs/torch/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ujjwal/miniconda3/envs/torch/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consid

In [17]:
print('response')
print(test.iloc[0,0])
print()
print('predicted')
print(test.iloc[0,1])

response
Hi Attica,This is a question I think a lot of people deal with...they feel confusion about why they can't forget about or get over (or stop connecting with) someone who they absolutely know isn't good for them. In your case, part of the problem is that he keeps trying to get back in touch with you. For some people, honestly, it's a game... to see how much power they have over you or it's their need to control you. If you don't want contact from this person, it's really important to give him clear messages about boundaries ("Don't contact me again"), and then ignore all of their communications. Any interactions or responses from you at all will feed their behaviour. Remind yourself why you don't want to be with them. I hear you doing that already when you say "he has nothing to offer me". That's great self-talk.But let's get back to the question of why we have a hard time letting go of people like this. Sometimes it's because we still hope they will change. Maybe we remember wh

In [18]:
torch.save(model.state_dict(), 'torchNLP.pth')